In [1]:
import time
from curl_cffi import requests as cureq
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}

In [3]:
#First check for the number of houses and pages to be scraped
url = f"https://www.immoweb.be/nl/search-results/huis/te-koop?countries=BE&page=1&orderBy=relevance"

resp = cureq.get(url, headers=headers, impersonate="chrome")

print(resp.status_code)

data = resp.json()

total_houses_1_page = int(data['range'].split('-')[1])

total_number_of_houses= data['totalItems']

number_of_pages = total_number_of_houses//(total_houses_1_page + 1)

print(f'Immoweb contains {total_number_of_houses} listings on {number_of_pages} pages')


200
Immoweb contains 9969 listings on 332 pages


In [4]:
#FIRST SCRAPE: Scrape function

def collect_data(pages: int, headers: dict):
    """
    This function collects data from the specified number of pages and returns a list.

    Args:
    pages (int): Number of pages to collect data from.
    headers (dict): HTTP headers to include in the requests.
    """
    if not headers:
        raise ValueError("Headers are required to make the request.")

    data_collection = []

    # Start the timer
    start_time = time.perf_counter()
    print(f"Start scraping {pages} pages...")

    # Loop through the pages and collect data
    for page in range(1, pages + 1):
        url = f"https://www.immoweb.be/nl/search-results/huis/te-koop?countries=BE&page={page}&orderBy=relevance"
        
        try:
            # Send the request
            print(f"Scraping page {page}...")
            resp = cureq.get(url, headers=headers, impersonate="chrome")
            resp.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

            # Try to parse the JSON response
            data = resp.json()
            data_collection.append(data)
            print(f"Page {page} scraped successfully.")
        
        except requests.exceptions.RequestException as req_err:
            # Handle network-related errors
            print(f"Network error on page {page}: {req_err}")
        except ValueError:
            # Handle JSON decoding errors
            print(f"Error decoding JSON on page {page}.")
        except Exception as e:
            # Catch any other exceptions
            print(f"An error occurred on page {page}: {e}")

    # Stop the timer and print the elapsed time
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Data scraping completed in {elapsed_time:.2f} seconds.")

    return data_collection


In [5]:
#FIRST SCRAPE: Parse function

def parse_data(listings: list):
    """
    Parses the listings data to extract relevant property details.

    Args:
    listings (list): List of raw listings data.

    Returns:
    all_properties (list): List of dictionaries with parsed property data.
    """
    all_properties = []

    for listing in listings:
        # Get the results list (default to an empty list if not found)
        results_list = listing.get('results', [])

        for result in results_list:
            # Extract property details using nested get() for safety
            property_details = result.get('property', {})
            location = property_details.get('location', {})
            transaction = result.get('transaction', {}).get('sale', {})

            property_data = {
                'id': result.get('id'),
                'type': property_details.get('type'),
                'subtype': property_details.get('subtype'),
                'country': location.get('country'),
                'region': location.get('region'),
                'locality_name': location.get('locality'),
                'locality_code': location.get('postalCode'),
                'street': location.get('street'),
                'number': location.get('number'),
                'latitude': location.get('latitude'),
                'longitude': location.get('longitude'),
                'bedroom_count': property_details.get('bedroomCount', "None"),
                'net_habitable_surface': property_details.get('netHabitableSurface'),
                'land_surface': property_details.get('landSurface'),
                'room_count': property_details.get('roomCount', "None"),
                'transaction_type': result.get('transaction', {}).get('type'),
                'sale_annuity': transaction.get('lifeAnnuity'),
                'price': transaction.get('price'),
                'old_price': transaction.get('oldPrice')
            }

            all_properties.append(property_data)

    print(f"Extracted {len(all_properties)} properties from data.")
    return all_properties


In [6]:
#Testing and starting the first scrape with the scrape function

listings = collect_data(number_of_pages, headers)


Start scraping 332 pages...
Scraping page 1...
Page 1 scraped successfully.
Scraping page 2...
Page 2 scraped successfully.
Scraping page 3...
Page 3 scraped successfully.
Scraping page 4...
Page 4 scraped successfully.
Scraping page 5...
Page 5 scraped successfully.
Scraping page 6...
Page 6 scraped successfully.
Scraping page 7...
Page 7 scraped successfully.
Scraping page 8...
Page 8 scraped successfully.
Scraping page 9...
Page 9 scraped successfully.
Scraping page 10...
Page 10 scraped successfully.
Scraping page 11...
Page 11 scraped successfully.
Scraping page 12...
Page 12 scraped successfully.
Scraping page 13...
Page 13 scraped successfully.
Scraping page 14...
Page 14 scraped successfully.
Scraping page 15...
Page 15 scraped successfully.
Scraping page 16...
Page 16 scraped successfully.
Scraping page 17...
Page 17 scraped successfully.
Scraping page 18...
Page 18 scraped successfully.
Scraping page 19...
Page 19 scraped successfully.
Scraping page 20...
Page 20 scraped succ

In [7]:
#Here i create a json file with one element of listings so we can have a look of the json data from one listing on the overview pages that was scraped during the first scrape
with open('listings_first_scrape_test.json', 'w') as f:
    json.dump(listings[0], f, indent=4)

In [8]:
#Parse all the data from the first scrape and put it in a list

all_properties = parse_data(listings)

Extracted 9960 properties from data.


In [9]:
#An overview of the data that was kept from one listing

all_properties[18]

{'id': 20250577,
 'type': 'HOUSE',
 'subtype': 'APARTMENT_BLOCK',
 'country': 'België',
 'region': 'Vlaanderen',
 'locality_name': 'SINT-KATELIJNE-WAVER',
 'locality_code': '2860',
 'street': 'IJzerenveld',
 'number': '4',
 'latitude': 51.0349503,
 'longitude': 4.5113723,
 'bedroom_count': 1,
 'net_habitable_surface': 43,
 'land_surface': 70,
 'room_count': None,
 'transaction_type': 'FOR_SALE',
 'sale_annuity': None,
 'price': 345000,
 'old_price': None}

In [10]:
#create a first listings list 

listings_1 = []  

for house in all_properties:
    listing_1 = {}  
    listing_1['id'] = house.get('id')
    listing_1['locality_name'] = house.get('locality_name')
    listing_1['Postal_code'] = house.get('locality_code')
    listing_1['Price'] = house.get('price')
    listing_1['Subtype'] = house.get('subtype')
    listing_1['Number_of_rooms'] = house.get('room_count')
    listing_1['Number_of_bedrooms'] = house.get('bedroom_count')
    listing_1['Living_area'] = house.get('net_habitable_surface')
    listing_1['sale_annuity'] = house.get('sale_annuity')
    listing_1['Type_of_sale'] = house.get('transaction_type')
    listing_1['street'] = house.get('street')
    listing_1['number'] = house.get('number')
    listing_1['latitude'] = house.get('latitude')
    listing_1['longitude'] = house.get('longitude')
    
    
    listings_1.append(listing_1)  

listings_1

    
    

[{'id': 20252354,
  'locality_name': 'RONSE',
  'Postal_code': '9600',
  'Price': 319000,
  'Subtype': 'HOUSE',
  'Number_of_rooms': None,
  'Number_of_bedrooms': 3,
  'Living_area': 125,
  'sale_annuity': None,
  'Type_of_sale': 'FOR_SALE',
  'street': 'Rotterij',
  'number': '148',
  'latitude': 50.744176,
  'longitude': 3.625722199999999},
 {'id': 20251003,
  'locality_name': 'GERAARDSBERGEN',
  'Postal_code': '9500',
  'Price': 299999,
  'Subtype': 'HOUSE',
  'Number_of_rooms': None,
  'Number_of_bedrooms': 3,
  'Living_area': 167,
  'sale_annuity': None,
  'Type_of_sale': 'FOR_SALE',
  'street': 'Pirrestraat',
  'number': '17',
  'latitude': 50.7521206,
  'longitude': 3.9254954},
 {'id': 20247788,
  'locality_name': 'Tielt (8700)',
  'Postal_code': '8700',
  'Price': None,
  'Subtype': 'HOUSE_GROUP',
  'Number_of_rooms': None,
  'Number_of_bedrooms': None,
  'Living_area': None,
  'sale_annuity': None,
  'Type_of_sale': 'FOR_SALE',
  'street': 'Kistestraat',
  'number': '13',
  'l

In [11]:
# Convert listings_1 to DataFrame and export to CSV
dataframe_first_scrape = pd.DataFrame(listings_1)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_2_test.csv"
dataframe_first_scrape.to_csv(csv_path, index=False)

print(f"Data from first scrape saved to {csv_path}")

Data from first scrape saved to C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_2_test.csv


In [12]:
## SECOND SCRAPE: This is the scrape of the individual house pages

print("Start second scraping")
    
data_collection_2 = []

for property in listings_1:
    
    base_url = "https://www.immoweb.be/nl/zoekertje/"
    property_url = base_url + str(property['id'])
    
    try:
        # Send request to property URL
        r = requests.get(property_url, headers=headers)
        r.raise_for_status()  
        
        # Parse the page content
        soup = BeautifulSoup(r.content, "html.parser")
        print(f"Page {property_url} scraped successfully.")
        
        # Try to extract the specific script tag
        script_tag = soup.select_one('div.classified script[type="text/javascript"]')
        
        if script_tag:
            # Process the script_tag here
            print("Script tag found and processed.")
        else:
            print(f"No script tag found on {property_url}")
    
    except Exception as e:
        # Print the error message and continue to the next property
        print(f"Error on page {property_url}: {e}")
        continue
    
    # Extract the JavaScript object
    js_content = script_tag.string
    
    # Find the start and end of the JSON object
    start = js_content.find('{')
    end = js_content.rfind('}') + 1
    
    # Extract and parse the JSON data
    json_data = json.loads(js_content[start:end])
    
    #append json_data to the list data_collection2
    data_collection_2.append(json_data)




Start second scraping
Page https://www.immoweb.be/nl/zoekertje/20252354 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20251003 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20247788 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20249506 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20247946 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20251611 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20249532 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20249501 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20252002 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/2

In [13]:
#Here i create a json file with the data for one individual listing/house that i scraped during the second scrape
with open('listings_second_scrape_test.json', 'w') as f:
    json.dump(data_collection_2[15], f, indent=4)

In [14]:
#parse en merge the data from data_collection_2 in listings_2
listings_2 = []

for house in data_collection_2:
    listing_2 = {}
    
    
    listing_2['id'] = house.get('id', None)
    
    
    property_details = house.get('property', {})

    listing_2['Open_fire'] = property_details.get('fireplaceExists', None)
    
    listing_2['Swimming_Pool'] = property_details.get('hasSwimmingPool', None)
    
    listing_2['hasTerrace'] = property_details.get('hasTerrace', None)
    
    listing_2['terraceSurface'] = property_details.get('terraceSurface', None)
    
    listing_2['hasGarden'] = property_details.get('hasGarden', None)
    
    listing_2['gardenSurface'] = property_details.get('gardenSurface', None)
    
    
    kitchen_details = property_details.get('kitchen', {})
    
    if isinstance(kitchen_details, dict):
        listing_2['Kitchen_type'] = kitchen_details.get('type', None)
    else:
        listing_2['Kitchen_type'] = None
        
    
    building_details = property_details.get('building',{})
    
    if isinstance(building_details, dict):
        listing_2['Number_of_facades'] = building_details.get('facadeCount', None)
    else:
        listing_2['Number_of_facades'] = None
    
    if isinstance(building_details, dict):
        listing_2['State_of_building'] = building_details.get('condition', None)
    else:
        listing_2['State_of_building'] = None
    
    
    transaction_details = house.get('transaction',{})
    
    
    sale_details = transaction_details.get('sale',{})
    
    if isinstance(sale_details, dict):
        listing_2['Furnished'] = sale_details.get('isFurnished', None)
    else:
        listing_2['Furnished'] = None
    
    if isinstance(sale_details, dict):
        listing_2['Starting_price'] = sale_details.get('hasStartingPrice', None)
    else:
        listing_2['Starting_price'] = None
    
    certificates = transaction_details.get('certificates', {})
    
    if isinstance(certificates, dict):
        listing_2['epc'] = certificates.get('epcScore', None)
    else:
        listing_2['epc'] = None
    
    listings_2.append(listing_2)
    
    

In [15]:
#Here i test one element (= one id/house) in listings_2
listings_2[24]

{'id': 20252127,
 'Open_fire': False,
 'Swimming_Pool': False,
 'hasTerrace': True,
 'terraceSurface': 13,
 'hasGarden': True,
 'gardenSurface': 41,
 'Kitchen_type': 'INSTALLED',
 'Number_of_facades': 2,
 'State_of_building': 'AS_NEW',
 'Furnished': False,
 'Starting_price': True,
 'epc': 'E'}

In [16]:
#Creating a dataframe of listings_2

dataframe_second_scrape = pd.DataFrame(listings_2)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_test2.csv"
dataframe_second_scrape.to_csv(csv_path, index=False)

print(f"Data from second scrape saved to {csv_path}")

Data from second scrape saved to C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_test2.csv


In [17]:
# Merge all the data (listings_1 & listings_2) on key id

merged_df = pd.merge(dataframe_first_scrape, dataframe_second_scrape, on='id', how='inner')

print(merged_df)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_merged_test.csv"
merged_df.to_csv(csv_path, index=False)

print(f"Merged data saved to {csv_path}")


            id          locality_name Postal_code     Price      Subtype  \
0     20252354                  RONSE        9600  319000.0        HOUSE   
1     20251003         GERAARDSBERGEN        9500  299999.0        HOUSE   
2     20247788           Tielt (8700)        8700       NaN  HOUSE_GROUP   
3     20249506                 Deinze        9800       NaN  HOUSE_GROUP   
4     20247946                 Ninove        9400       NaN  HOUSE_GROUP   
...        ...                    ...         ...       ...          ...   
9984  20157543           Tielt (8700)        8700  295000.0        HOUSE   
9985  20140297              Vilvoorde        1800  299000.0        HOUSE   
9986  20200525                   AMAY        4540  299000.0        HOUSE   
9987  20125321  Saint-Hubert, Mirwart        6870  299000.0        HOUSE   
9988  20207583                 Izegem        8870  299000.0        HOUSE   

      Number_of_rooms  Number_of_bedrooms  Living_area sale_annuity  \
0               

In [18]:
#Showing the first 50 rows of the merged dataframe
merged_df.head(50)

,id,locality_name,Postal_code,Price,Subtype,Number_of_rooms,Number_of_bedrooms,Living_area,sale_annuity,Type_of_sale,...,hasTerrace,terraceSurface,hasGarden,gardenSurface,Kitchen_type,Number_of_facades,State_of_building,Furnished,Starting_price,epc
0,20252354,RONSE,9600,319000.0,HOUSE,NaN,3.0,125.0,None,FOR_SALE,...,True,NaN,True,370.0,None,4.0,TO_RENOVATE,None,None,F
1,20251003,GERAARDSBERGEN,9500,299999.0,HOUSE,NaN,3.0,167.0,None,FOR_SALE,...,True,NaN,True,895.0,INSTALLED,2.0,GOOD,None,None,D
2,20247788,Tielt (8700),8700,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
3,20249506,Deinze,9800,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
4,20247946,Ninove,9400,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
5,20251611,Gavere,9890,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
6,20249532,Sint-Lievens-Houtem,9520,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
7,20249501,Gent Mariakerke,9030,NaN,HOUSE_GROUP,NaN,NaN,NaN,None,FOR_SALE,...,None,NaN,None,NaN,None,NaN,None,None,None,None
8,20252002,MECHELEN,2800,275000.0,HOUSE,NaN,3.0,154.0,None,FOR_SALE,...,True,10.0,True,20.0,SEMI_EQUIPPED,2.0,TO_RENOVATE,None,None,E
9,20252352,Gooik,1755,295000.0,HOUSE,NaN,3.0,172.0,None,FOR_SALE,...,True,NaN,True,300.0,None,3.0,TO_RENOVATE,None,None,F


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  9989 non-null   int64  
 1   locality_name       9989 non-null   object 
 2   Postal_code         9989 non-null   object 
 3   Price               9334 non-null   float64
 4   Subtype             9989 non-null   object 
 5   Number_of_rooms     3327 non-null   float64
 6   Number_of_bedrooms  9350 non-null   float64
 7   Living_area         9164 non-null   float64
 8   sale_annuity        16 non-null     object 
 9   Type_of_sale        9989 non-null   object 
 10  street              9701 non-null   object 
 11  number              9620 non-null   object 
 12  latitude            9699 non-null   float64
 13  longitude           9699 non-null   float64
 14  Open_fire           9989 non-null   bool   
 15  Swimming_Pool       3855 non-null   object 
 16  hasTer